In [5]:
import time
import numpy as np
import torch
torch.autograd.set_grad_enabled(False)
# torch.cuda.set_device(0)

In [ ]:
!git clone --depth https://github.com/Hzzone/torch_clustering
!cd torch_clustering && pip install -e .

In [ ]:
from torch_clustering import PyTorchKMeans, FaissKMeans, PyTorchGaussianMixture, evaluate_clustering

In [ ]:
# CIFAR-10 features 256d
# https://drive.google.com/file/d/1N3REtPr-A__KOtzRZenFSvRIEufW9j3A/view
# ImageNet features 256d
# https://drive.google.com/file/d/1UcUWw9o6Y6SrSFXOkNTVjCoEik79oUAo/view

In [ ]:
!gdown --help

In [12]:
# !gdown --id 1N3REtPr-A__KOtzRZenFSvRIEufW9j3A -O cifar10_features.pth
!gdown --id 1UcUWw9o6Y6SrSFXOkNTVjCoEik79oUAo -O imagenet_features.pth

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1UcUWw9o6Y6SrSFXOkNTVjCoEik79oUAo
To: /content/imagenet_features.pth
100% 1.32G/1.32G [00:17<00:00, 75.3MB/s]


In [ ]:
features_path = './cifar10_features.pth'
# features_path = '.imagenet_features.pth'
data = torch.load(features_path, map_location='cpu')
features, labels = data[:, :-1], data[:, -1]
features, labels = features.cuda().float(), labels.cuda().long()
features_numpy, labels_numpy = features.cpu().numpy(), labels.cpu().numpy()

In [ ]:
kwargs = {
    'metric': 'cosine',  # euclidean if not l2_normalize
    'distributed': False,
    'random_state': 0,
    'n_clusters': int(labels_numpy.max() + 1),
    'verbose': False
}

In [ ]:
clustering_model = PyTorchKMeans(init='k-means++', max_iter=300, tol=1e-4, **kwargs)
psedo_labels = clustering_model.fit_predict(features)
evaluate_clustering(labels.cpu().numpy(), psedo_labels.cpu().numpy(), eval_metric=['nmi', 'acc'], phase='train')

{'train_nmi': 0.8551917177300482, 'train_acc': 0.92368}

In [ ]:
clustering_model = FaissKMeans(n_init=5, max_iter=50, **kwargs)
psedo_labels = clustering_model.fit_predict(features_numpy, device=0)
evaluate_clustering(labels.cpu().numpy(), psedo_labels.cpu().numpy(), eval_metric=['nmi', 'acc'], phase='train')

{'train_nmi': 0.855235833704221, 'train_acc': 0.92358}

In [ ]:
clustering_model = PyTorchGaussianMixture(covariance_type='diag', reg_covar=1e-6, init='k-means++', **kwargs)
psedo_labels = clustering_model.fit_predict(features)
evaluate_clustering(labels.cpu().numpy(), torch.argmax(psedo_labels, dim=1).cpu().numpy(), eval_metric=['nmi', 'acc'])

{'train_nmi': 0.8559683572879772, 'train_acc': 0.92388}

In [ ]:
n_times = 10
all_times = []
for i in range(n_times):
    st = time.time()
    PyTorchKMeans(init='k-means++', max_iter=300, tol=1e-4, **kwargs).fit_predict(features)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)

(2.2640299797058105, 0.0804519734107353)

In [ ]:
all_times = []
for i in range(n_times):
    st = time.time()
    FaissKMeans(n_init=5, max_iter=50, **kwargs).fit_predict(features_numpy, device=0)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)

(8.365153431892395, 0.6574485328135397)

In [ ]:
all_times = []
for i in range(n_times):
    st = time.time()
    PyTorchGaussianMixture(covariance_type='diag', reg_covar=1e-6, init='k-means++', **kwargs).fit_predict(features)
    all_times.append(time.time() - st)
np.mean(all_times), np.std(all_times)